In [1]:
import json, os, math, warnings, pandas as pd, numpy as np
from pathlib import Path

# --- Targets A–C (from your earlier work) ---
TARGETS = [
    # name,           TIC,         sectors,          period_d,        t0_btjd,           duration_hr,  depth_ppm_guess
    ("Target A", "119584412", [22, 49], 16.02749976, 1908.046283,      6.53,             450),
    ("Target B",  "37749396", [3, 42, 70], 13.47582381, 1392.306006,   3.00,             120),
    ("Target C", "311183180", [5, 31],   9.34849077,  2149.633454,     2.80,            2200),
]

FIGDIR = Path("figures")
RESDIR = Path("results")
FIGDIR.mkdir(exist_ok=True)
RESDIR.mkdir(exist_ok=True)

def load_refined_ephem_if_exists(tic):
    """Prefer your saved refined ephemeris JSON if present."""
    f = next(iter(RESDIR.glob(f"TIC{tic}_refined_ephemeris*.json")), None)
    if f and f.exists():
        with open(f) as fp:
            d = json.load(fp)
        P = float(d.get("P", d.get("period", np.nan)))
        T0 = float(d.get("T0", d.get("t0", np.nan)))
        return P, T0
    return None

def depthppm_to_rprs(depth_ppm):
    # box approx: depth ≈ (Rp/R★)^2  => Rp/R★ = sqrt(depth)
    return np.sqrt(max(depth_ppm, 0) / 1e6)

def btjd_to_bjdtdb(btjd):
    # TRICERATOPS expects BJD_TDB; BTJD = BJD_TDB - 2457000
    return 2457000.0 + btjd

In [2]:
import triceratops as tri

def run_triceratops(tic, period_d, t0_btjd, duration_hr, sectors=None, depth_ppm_guess=None):
    # Build target object (will handle Gaia/TIC/neighbor info internally)
    targ = tri.Target(ticid=int(tic))
    
    # Observing sectors (optional but helps)
    if sectors:
        targ.get_sectors()

    # Convert inputs to what TRICERATOPS expects
    t0_bjd = btjd_to_bjdtdb(t0_btjd)
    rprs_guess = depthppm_to_rprs(depth_ppm_guess or 0)

    # Compute false-positive probabilities at the candidate ephemeris
    # (TRICERATOPS uses period, duration (days), and T0 in BJD_TDB)
    duration_d = duration_hr / 24.0
    
    # Core call (API stable in v1.x): returns dict with 'FPP' and 'NFPP'
    res = targ.transitFPP(
        period=period_d,
        rprs=rprs_guess if rprs_guess > 0 else None,
        t0=t0_bjd,
        dur=duration_d,
        # If you have stellar params you can pass them; otherwise TRICERATOPS queries TIC/Gaia:
        # rstar=..., mstar=..., teff=..., logg=..., feh=...
        # You can also set n=2_000_000 for more thorough runs later; keep it lighter first.
    )
    # Extract headline numbers
    fpp   = float(res.get("FPP"))
    nfpp  = float(res.get("NFPP"))  # Nearby-FPP (blend probability)
    ret = {
        "TIC": tic,
        "P_d": period_d,
        "T0_BTJD": t0_btjd,
        "dur_hr": duration_hr,
        "depth_ppm_guess": depth_ppm_guess,
        "FPP": fpp,
        "NearbyFPP": nfpp,
    }
    return ret, res

In [3]:
rows = []
raw_outputs = {}

for name, tic, sectors, P, T0, DUR, DEPTH in TARGETS:
    # prefer your refined files if present
    maybe = load_refined_ephem_if_exists(tic)
    if maybe:
        P, T0 = maybe
    print(f"[TRICERATOPS] {name} TIC {tic}  P={P:.8f} d  T0(BTJD)={T0:.6f}  dur={DUR:.2f} h")
    try:
        row, raw = run_triceratops(
            tic=tic, period_d=P, t0_btjd=T0, duration_hr=DUR,
            sectors=sectors, depth_ppm_guess=DEPTH
        )
        rows.append({"Name": name, **row})
        raw_outputs[tic] = raw
    except Exception as e:
        print(f"  !! ERROR on TIC {tic}: {e}")
        rows.append({"Name": name, "TIC": tic, "P_d": P, "T0_BTJD": T0, "dur_hr": DUR,
                     "depth_ppm_guess": DEPTH, "FPP": np.nan, "NearbyFPP": np.nan})

df = pd.DataFrame(rows)
display(df)
df.to_csv(RESDIR / "Table2_triceratops_AC.csv", index=False)

# Save full JSON blobs too for provenance
with open(RESDIR / "Table2_triceratops_AC_raw.json", "w") as fp:
    json.dump(raw_outputs, fp, indent=2)
print("Saved:", RESDIR / "Table2_triceratops_AC.csv")

[TRICERATOPS] Target A TIC 119584412  P=16.02749976 d  T0(BTJD)=1908.046283  dur=6.53 h
  !! ERROR on TIC 119584412: module 'triceratops' has no attribute 'Target'
[TRICERATOPS] Target B TIC 37749396  P=13.47582381 d  T0(BTJD)=1392.306006  dur=3.00 h
  !! ERROR on TIC 37749396: module 'triceratops' has no attribute 'Target'
[TRICERATOPS] Target C TIC 311183180  P=9.34853858 d  T0(BTJD)=2149.632747  dur=2.80 h
  !! ERROR on TIC 311183180: module 'triceratops' has no attribute 'Target'


,Name,TIC,P_d,T0_BTJD,dur_hr,depth_ppm_guess,FPP,NearbyFPP
0,Target A,119584412,16.027500,1908.046283,6.53,450,NaN,NaN
1,Target B,37749396,13.475824,1392.306006,3.00,120,NaN,NaN
2,Target C,311183180,9.348539,2149.632747,2.80,2200,NaN,NaN


Saved: results/Table2_triceratops_AC.csv


In [4]:
def fmt_pct(x):
    if pd.isna(x): return "—"
    return f"{100*x:.3f}%"

md_lines = ["| Target | TIC | P (d) | T0 (BTJD) | dur (h) | FPP | Nearby-FPP |",
            "|---|---:|---:|---:|---:|---:|---:|"]
for r in df.itertuples():
    md_lines.append(
        f"| {r.Name} | {r.TIC} | {r.P_d:.6f} | {r.T0_BTJD:.6f} | {r.dur_hr:.2f} | {fmt_pct(r.FPP)} | {fmt_pct(r.NearbyFPP)} |"
    )
md = "\n".join(md_lines)
print(md)
with open(RESDIR / "Table2_triceratops_AC.md","w") as f:
    f.write(md)
print("Saved:", RESDIR / "Table2_triceratops_AC.md")

| Target | TIC | P (d) | T0 (BTJD) | dur (h) | FPP | Nearby-FPP |
|---|---:|---:|---:|---:|---:|---:|
| Target A | 119584412 | 16.027500 | 1908.046283 | 6.53 | — | — |
| Target B | 37749396 | 13.475824 | 1392.306006 | 3.00 | — | — |
| Target C | 311183180 | 9.348539 | 2149.632747 | 2.80 | — | — |
Saved: results/Table2_triceratops_AC.md


In [8]:
import numpy as np, pandas as pd, lightkurve as lk
import triceratops.triceratops as tr
from pathlib import Path

RESULTS = Path("results")
RESULTS.mkdir(exist_ok=True)

def load_spoc_pdcsap(tic, sectors, bitmask=175):
    """Return a stitched PDCSAP LightCurve for TIC over given sectors."""
    lcs = []
    for s in sectors:
        try:
            sr = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC", sector=s)
            if len(sr) == 0:
                print(f"  .. no SPOC LCF for TIC {tic} S{s}")
                continue
            lcf = sr.download()
            if lcf is None:
                print(f"  .. download failed for TIC {tic} S{s}")
                continue
            lc = lcf.PDCSAP_FLUX
            # quality mask (SPOC bitmask=175); then simple cleanup & normalize
            qmask = lk.utils.SPOCQualityFlags.create_quality_mask(lc.quality, bitmask=bitmask)
            lc = lc[qmask].remove_nans().normalize()
            lcs.append(lc)
        except Exception as e:
            print(f"  .. error on TIC {tic} S{s}: {e}")
    if not lcs:
        return None
    return lk.LightCurveCollection(lcs).stitch()

In [27]:
# === TRICERATOPS Table 2 runner (A/B/C) ===
# Requirements already in your env: triceratops==1.0.x, lightkurve>=2, astroquery, numpy<=1.23.* or shim below

import os, time, json, warnings
import numpy as np, pandas as pd
import lightkurve as lk
from lightkurve import search_lightcurve, search_lightcurvefile
from lightkurve.lightcurve import TessLightCurve
import triceratops.triceratops as tr

# -- NumPy alias shim for older codepaths (safe no-op if present)
for _n, _t in (("bool", bool), ("int", int), ("float", float)):
    if not hasattr(np, _n):
        setattr(np, _n, _t)

# Quieten benign warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="The PDCSAP_FLUX function is deprecated")

os.makedirs("results", exist_ok=True)

targets = [
    # Name, TIC, P_d, T0_BTJD, dur_hr, sectors
    ("Target A", 119584412, 16.02749976, 1908.046283, 6.53, [22, 49]),
    ("Target B",  37749396, 13.47582381, 1392.306006, 3.00, [ 3, 42, 70]),
    ("Target C", 311183180,  9.34853858, 2149.632747, 2.80, [ 5, 31]),
]

def fetch_spoc_pdcsap_stitched(tic, sectors):
    """Return a stitched, normalized, NaN-free TessLightCurve (PDCSAP) for TIC+sectors."""
    lcs = []
    for s in sectors:
        try:
            # prefer modern API
            sr = search_lightcurve(f"TIC {tic}", mission="TESS", author="SPOC", sector=s)
            lcf = sr.download()
        except Exception:
            # graceful fallback to legacy search
            sr = search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC", sector=s)
            lcf = sr.download()
        if lcf is None:
            continue
        # Accept either LC or LCF object
        lc = getattr(lcf, "PDCSAP_FLUX", None) or lcf
        try:
            lc = lc.remove_nans().remove_outliers(sigma=6.0)
        except Exception:
            lc = lc.remove_nans()
        lcs.append(lc)
    if not lcs:
        return None
    try:
        lcc = lk.LightCurveCollection(lcs).stitch()
    except Exception:
        lcc = lcs[0]
        for lc in lcs[1:]:
            lcc = lcc.append(lc)
    return lcc.normalize().remove_nans()

def build_transit_window(lc, P_d, T0_btjd, dur_hr, half_windows=1.5):
    """Return arrays in ±(half_windows*dur) around phase=0."""
    dur_days = dur_hr/24.0
    t = lc.time.value
    f = lc.flux.value
    phase = ((t - T0_btjd + 0.5*P_d) % P_d) - 0.5*P_d
    m = np.isfinite(f) & (np.abs(phase) <= half_windows*dur_days)
    if not np.any(m):
        return None
    return phase[m], f[m], dur_days

def estimate_depth_ppm(t_rel, f_rel, dur_days):
    """Robust depth estimate near mid-transit; returns depth in ppm (>= 10 ppm)."""
    # central window = ±0.6*dur; baseline = outside ±1.2*dur (within the clipped window)
    cen = np.abs(t_rel) <= 0.6*dur_days
    oot = np.abs(t_rel) >= 1.2*dur_days
    if not cen.any() or not oot.any():
        # fallback: use entire window vs. median=1
        depth_frac = max(0.0, 1.0 - float(np.nanmedian(f_rel)))
    else:
        depth_frac = max(0.0, float(np.nanmedian(f_rel[oot]) - np.nanmedian(f_rel[cen])))
    depth_ppm = max(10.0, depth_frac*1e6)  # keep strictly positive & ≥10 ppm
    return depth_ppm

def bin_to_minute(time_days, flux):
    """Return a ~1-min binned TessLightCurve for speed."""
    span = float(np.nanmax(time_days) - np.nanmin(time_days))
    one_min = 1.0/24.0/60.0
    binsize = max(one_min, span/200.0)
    tlc = TessLightCurve(time=time_days, flux=flux,
                         flux_err=np.full_like(flux, float(np.nanstd(flux) or 1e-4)))
    try:
        return tlc.bin(time_bin_size=binsize)
    except Exception:
        return tlc

def with_mast_retry(fn, tries=3, base_wait=3):
    """Retry wrapper for MAST/neighbor calls that occasionally 404."""
    for k in range(tries):
        try:
            return fn()
        except Exception as e:
            err = str(e)
            if k == tries-1:
                raise
            time.sleep(base_wait*(2**k))

rows = []
for name, tic, P, T0, dur_hr, sectors in targets:
    print(f"[TRICERATOPS] {name} TIC {tic}  P={P:.8f} d  T0={T0:.6f}  dur={dur_hr:.2f} h")

    lc = fetch_spoc_pdcsap_stitched(tic, sectors)
    if lc is None or len(lc.time) == 0:
        print("  !! No SPOC PDCSAP; skipping")
        rows.append([name, tic, P, T0, dur_hr, np.nan, np.nan])
        continue

    # gentle de-trend (~1 day window at 2-min cadence)
    try:
        lc = lc.flatten(window_length=721, polyorder=2)
    except Exception:
        pass

    built = build_transit_window(lc, P, T0, dur_hr)
    if built is None:
        print("  !! No points in predicted window; skipping")
        rows.append([name, tic, P, T0, dur_hr, np.nan, np.nan])
        continue
    t_rel, f_rel, dur_days = built
    depth_ppm = estimate_depth_ppm(t_rel, f_rel, dur_days)
    print(f"  .. using tdepth ≈ {depth_ppm:.0f} ppm")

    tlc = bin_to_minute(t_rel, f_rel)
    tqc_time = tlc.time.value
    tqc_flux = tlc.flux.value
    tqc_ferr = float(np.nanmedian(tlc.flux_err.value)) if tlc.flux_err is not None else float(np.nanstd(tqc_flux) or 1e-4)

    try:
        tgt = tr.target(ID=tic, sectors=sectors)
        # Ensure star field/apertures are initialized; allow TRICERATOPS defaults
        def _prep():
            # plot_field triggers neighbor fetch & caching in 1.0.x; we discard the plot
            try:
                tgt.plot_field(sector=sectors[0])
            except Exception:
                pass
            # per docs, set the transit depth BEFORE calc_probs
            # (calc_depths can accept None for apertures to use a centered 5x5)  [oai_citation:1‡triceratops.readthedocs.io](https://triceratops.readthedocs.io/en/latest/user/api.html?utm_source=chatgpt.com)
            tgt.calc_depths(tdepth=depth_ppm, all_ap_pixels=None)
        with_mast_retry(_prep)

        # Now run the probabilities (documented signature)  [oai_citation:2‡triceratops.readthedocs.io](https://triceratops.readthedocs.io/en/latest/user/api.html?utm_source=chatgpt.com)
        tgt.calc_probs(time=tqc_time, flux_0=tqc_flux, flux_err_0=tqc_ferr,
                       P_orb=P, verbose=0, N=200000, parallel=False)

        fpp, nfpp = float(tgt.FPP), float(tgt.NFPP)
        print(f"  -> FPP={fpp:.4f}  Nearby-FPP={nfpp:.4f}")

        # Save per-scenario table if available
        try:
            with open(f"results/TIC{tic}_triceratops_probs.json","w") as fh:
                json.dump(tgt.probs, fh, indent=2)
        except Exception:
            pass

    except Exception as e:
        print(f"  !! TRICERATOPS failed after retries: {e}")
        fpp, nfpp = np.nan, np.nan

    rows.append([name, tic, P, T0, dur_hr, fpp, nfpp])

# ---- Write Table 2 (CSV + Markdown)
df = pd.DataFrame(rows, columns=["Name","TIC","P_d","T0_BTJD","dur_hr","FPP","NearbyFPP"])
csv_path = "results/Table2_triceratops_AC.csv"
md_path  = "results/Table2_triceratops_AC.md"
df.to_csv(csv_path, index=False)

with open(md_path, "w") as f:
    f.write("| Target | TIC | P (d) | T0 (BTJD) | dur (h) | FPP | Nearby-FPP |\n")
    f.write("|---|---:|---:|---:|---:|---:|---:|\n")
    for _, r in df.iterrows():
        f.write(f"| {r['Name']} | {int(r['TIC'])} | {r['P_d']:.6f} | {r['T0_BTJD']:.6f} | {r['dur_hr']:.2f} | "
                f"{'—' if pd.isna(r['FPP']) else f'{r['FPP']:.4f}'} | "
                f"{'—' if pd.isna(r['NearbyFPP']) else f'{r['NearbyFPP']:.4f}'} |\n")

print(f"Saved: {csv_path}\nSaved: {md_path}")

SyntaxError: f-string: f-string: unmatched '[' (3876342511.py, line 182)